In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'SUN'
path = 'E:/Sushree/Dataset/data/xlsa17/data/'

res101 = scipy.io.loadmat(path + dataset + '/res101.mat')
att_splits = scipy.io.loadmat(path + dataset + '/att_splits.mat')

In [3]:
signature = att_splits['att']
#signature = att_splits['original_att']
#signature = signature/100
print(signature.shape) #(102, 717)

attribute = signature.transpose()
attribute[attribute<0] = 0
print(attribute, attribute.shape)#(102, 717)

(102, 717)
[[0.         0.04386302 0.04386302 ... 0.0087726  0.02631781 0.0087726 ]
 [0.00725916 0.3411804  0.31214377 ... 0.00725916 0.02903663 0.00725916]
 [0.         0.03359921 0.09239783 ... 0.0083998  0.04199902 0.04199902]
 ...
 [0.         0.04857575 0.04857575 ... 0.00971515 0.194303   0.00971515]
 [0.         0.08417375 0.04208687 ... 0.01402896 0.03507239 0.02805792]
 [0.03236677 0.         0.01078892 ... 0.03236677 0.07552247 0.        ]] (717, 102)


In [4]:
import gensim.downloader as api
import pandas as pd
print('Load pretrain w2v model')

#model_name = 'word2vec-google-news-300' # length = 300
#model = api.load(model_name)

import transformers
from transformers import BertTokenizer, TFBertModel, BartTokenizer, TFBartModel, GPT2Tokenizer, TFGPT2Model
from transformers import OpenAIGPTTokenizer, TFOpenAIGPTModel

# Bert model
#---------------------------------------------------------------------------
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # length 768
#model = TFBertModel.from_pretrained("bert-base-uncased")

#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased') # 1024
#model = TFBertModel.from_pretrained("bert-large-uncased")

# Bart model
#---------------------------------------------------------------------------
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-base') # length 768
#model = TFBartModel.from_pretrained("facebook/bart-base")

#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large') # length 1024
#model = TFBartModel.from_pretrained("facebook/bart-large")

# GPT2 model
#---------------------------------------------------------------------------
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # length 768
model = TFGPT2Model.from_pretrained('gpt2')

# GPT model
#---------------------------------------------------------------------------
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')# length 768
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')

Load pretrain w2v model


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [5]:
#tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
#model = TFOpenAIGPTModel.from_pretrained('openai-gpt')
#encoded_input = tokenizer('sushree', return_tensors='tf')
#print(encoded_input)
#length = encoded_input.input_ids.shape[1]
#print(length)

In [6]:
#print(tokenizer)
#out = model(encoded_input).last_hidden_state[:, 2, :]
#print(out.shape)

In [7]:
#import scipy.io as sio

#%%
#replace_word = [('rockstone','rock stone'),('dirtsoil','dirt soil'),('man-made','man made'),('sunsunny','sun sunny'),
#                ('electricindoor','electric indoor'),('semi-enclosed','semi enclosed'),('far-away','faraway'), 
#                ('wood (not part of a tree)', 'wood')] # for SUN
#

#file_path = 'E:/Sushree/Dataset/SUNAttributeDB_Images/attribute/attributes.mat'
#matcontent = sio.loadmat(file_path)
#des = matcontent['attributes'].flatten()

#%%
#df = pd.DataFrame()
#new_des = [''.join(i.item().split('/')) for i in des]

#%% replace out of dictionary words
#for pair in replace_word:
#    for idx,s in enumerate(new_des):
#        new_des[idx]=s.replace(pair[0],pair[1])
#print('Done replace OOD words')

#df['new_des']=new_des
#df.to_csv('E:/Sushree/Dataset/SUNAttributeDB_Images/attribute/new_des.csv')
#print('Done preprocessing attribute des')

In [8]:

dim_w2v = 768


#%% # For SUN

file_path = 'E:/Sushree/Dataset/SUNAttributeDB_Images/attribute/new_des.csv'
df = pd.read_csv(file_path)
des = df['new_des'].values

import pickle


# for word2vec
# -----------------------------------------------------------------------------------
#counter_err = 0
#
#all_w2v = []
#for s in des:
#    print(s)
#    words = s.split(' ')
#    if words[-1] == '':     #remove empty element
#        words = words[:-1]
#    w2v = np.zeros(dim_w2v)
#    for w in words:
#        try:
#            w2v += model[w]
#        except Exception as e:
#            print(e)
#            counter_err += 1
#    w2v = w2v / len(words)  
#    all_w2v.append(w2v[np.newaxis,:])
#    
#print('counter_err ',counter_err)

#w2v_att = np.concatenate(all_w2v,axis=0)
#print(w2v_att, w2v_att.shape)

# for Bert attributes
# -----------------------------------------------------------------------------------
counter = 0
w2v_att = np.zeros((signature.shape[0], dim_w2v))
for s in des:
    print(s)
    w2v = np.zeros(dim_w2v)
    encoded_input = tokenizer(s, return_tensors='tf')
    length = encoded_input.input_ids.shape[1]
    #print(length)
    #for i in range(length-2): # for Bert, Bart
    for i in range(length): # for GPT2
        #w2v = w2v + model(encoded_input).last_hidden_state[:, i+1, :] # for Bert, Bart
        w2v = w2v + model(encoded_input).last_hidden_state[:, i, :] # for GPT2
        #print(model(encoded_input).last_hidden_state[:, i+1, :][:,1:2])
        #print(w2v[:, 1:2])
        #print(w2v.shape)

    #w2v = w2v / (length - 2)# for Bert, Bart
    w2v = w2v / length # for GPT2
    #print(w2v[:, 1:2])
    w2v_att[counter] = w2v
    counter = counter + 1

print('counter ',counter)
print(w2v_att, w2v_att.shape)
# -----------------------------------------------------------------------------------


attribute_new = np.einsum('ij,jl->ijl', attribute, w2v_att)

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], attribute_new.shape[1]* attribute_new.shape[2]])
print(attribute_new, attribute_new.shape)

#attribute_new[attribute_new<0] = 0
#print(attribute_new, attribute_new.shape)

sailing boating
driving
biking
transporting things or people
sunbathing
vacationing touring
hiking
climbing
camping
reading
studying learning
teaching training
research
diving
swimming
bathing
eating
cleaning
socializing
congregating
waiting in line queuing
competing
sports
exercise
playing
gaming
spectating being in an audience
farming
constructing building
shopping
medical activity
working
using tools
digging
conducting business
praying
fencing
railing
wire
railroad
trees
grass
vegetation
shrubbery
foliage
leaves
flowers
asphalt
pavement
shingles
carpet
brick
tiles
concrete
metal
paper
wood
vinyl linoleum
rubber plastic
cloth
sand
rock stone
dirt soil
marble
glass
waves surf
ocean
running water
still water
ice
snow
clouds
smoke
fire
natural light
direct sun sunny
electric indoor lighting
aged worn
glossy
matte
sterile
moist damp
dry
dirty
rusty
warm
cold
natural
man made
open area
semi enclosed area
enclosed area
faraway horizon
no horizon
rugged scene
mostly vertical components
most

In [9]:

# total number of instances or images = 14340: ranges from 0 to 14339

trainval_loc = np.squeeze(att_splits['trainval_loc']-1) # -1: to consider the overflow problem
print(np.unique(trainval_loc), np.max(np.unique(trainval_loc))) # smallest location: 2, largest location 37321

labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 14340 x 1

print('unique_labels', np.unique(labels), np.unique(labels).shape)# class labels range from 1 to 717, 717 classes

# get the labels for trainval, test seen and test unseen sets

labels_trainval = labels[trainval_loc]
print('labels_trainval', labels_trainval, labels_trainval.shape)

unique_labels_trainval = np.unique(labels_trainval) # labels min:1 max:717
print('unique_labels_trainval', unique_labels_trainval, unique_labels_trainval.shape)# 645 classes



[    1     2     4 ... 14337 14338 14339] 14339
labels [[  1]
 [  1]
 [  1]
 ...
 [717]
 [717]
 [717]] (14340, 1)
unique_labels [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 

In [10]:
X_features = res101['features']

# locations are already subtracted by 1, so they range from 0 to 14339
trainval_vec = X_features[:, trainval_loc].transpose()
print("Features for trainval:", trainval_vec.shape) #(10320, 2048)

# attribute is defined for all 717 classes, so we cant use locations directly, instead we have to use labels 
# that range from 1 to 717, so we have to subtract 1

trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (10320, 78336)


Features for trainval: (10320, 2048)
[[ 0.          0.          0.         ...  0.00616721 -0.00829422
  -0.00335526]
 [ 0.          0.          0.         ...  0.00616721 -0.00829422
  -0.00335526]
 [ 0.          0.          0.         ...  0.00616721 -0.00829422
  -0.00335526]
 ...
 [ 0.          0.          0.         ...  0.00289653 -0.00389551
  -0.00157585]
 [ 0.          0.          0.         ...  0.00289653 -0.00389551
  -0.00157585]
 [ 0.          0.          0.         ...  0.00289653 -0.00389551
  -0.00157585]] (10320, 78336)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad
import keras.backend as K

attribute_shape_new = attribute_new.shape[1]
print(attribute_shape_new)

# define model for attribute transformation

inputt = Input(shape = attribute_shape_new)
hidden = Dense(4096, name="layer1", activation='linear')(inputt)
output = Dense(2048, name="layer3", activation='linear')(hidden)

model_transform_attribute = Model(inputs = inputt, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model_transform_attribute.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

for layer in model_transform_attribute.layers:
        layer.trainable = False
        
model_transform_attribute.summary()

attribute_new = model_transform_attribute.predict(attribute_new)
print(attribute_new, attribute_new.shape)


trainval_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    trainval_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(trainval_attributes, trainval_attributes.shape)# (23527, 85)

78336
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 78336)]           0         
                                                                 
 layer1 (Dense)              (None, 4096)              320868352 
                                                                 
 layer3 (Dense)              (None, 2048)              8390656   
                                                                 
Total params: 329,259,008
Trainable params: 0
Non-trainable params: 329,259,008
_________________________________________________________________
23/23 [==============================] - 0s 6ms/step
[[-0.25936854  1.2479748   1.4921916  ...  0.91449904  0.08143608
   0.26827574]
 [-0.14264192  1.3863513   1.7471155  ...  0.38474512  0.27519003
   0.47888482]
 [ 0.16885011  1.0341734   2.148685   ...  0.70529246  0.7445255
   0.8264952 ]
 ...
 [ 0.

In [12]:
trainval_input1 = np.reshape(trainval_vec, [trainval_vec.shape[0], 1, 1, trainval_vec.shape[1]])
print(trainval_input1.shape)

trainval_input2 = np.reshape(trainval_attributes, [trainval_attributes.shape[0], 1, 1, trainval_attributes.shape[1]])
print(trainval_input2.shape)

(10320, 1, 1, 2048)
(10320, 1, 1, 2048)


In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence

X_train, X_val, y_train, y_val = train_test_split(trainval_input1, trainval_input2, test_size = 0.2, random_state = 42)

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 8
train_gen = DataGenerator(X_train, y_train, batch_size)   
val_gen = DataGenerator(X_val, y_val, batch_size)

In [14]:
# embedding model
# -----------------------------------------------------------------------------------
inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
hidden1 = Conv2D(2048, 1, name="layer1", activation='relu')(inputt)
output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

embedding_model = Model(inputs = inputt, outputs = output2)

# embedding model variants
# -----------------------------------------------------------------------------------

#inputt = Input(shape = (1, 1, trainval_vec.shape[1]))
#hidden1 = Conv2D(1024, 1, name="layer1", activation='relu')(inputt)
#output1 = Conv2D(1024, 1, name="layer2", activation='relu')(hidden1)
#hidden2 = Conv2D(1024, 1, name="layer3", activation='relu')(output1)
#output2 = Conv2D(trainval_attributes.shape[1], 1, name="layer4", activation='linear')(hidden2)

#embedding_model = Model(inputs = inputt, outputs = output2)


adam = Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
embedding_model.compile(adam, loss = tf.keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.AUTO), metrics = ['accuracy'])
#embedding_model.compile(adam, loss = cosine_loss, metrics = ['accuracy'])

embedding_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
 layer3 (Conv2D)             (None, 1, 1, 1024)        1049600   
                                                                 
 layer4 (Conv2D)             (None, 1, 1, 2048)        2099200   
                                                                 
Total params: 9,443,328
Trainable params: 9,443,328
Non-trainable params: 0
_________________________________________________________________


In [15]:

#CosineSimilarity is a number between -1 and 1. When it is a negative number between -1 and 0, 0 indicates orthogonality and 
#values closer to -1 indicate greater similarity. The values closer to 1 indicate greater dissimilarity. 

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/SUN/with_LE/Embedding Model/'
name = 'embedding_model_GPT2att_2048_1024_1024_SUN_200eph_adam_cos_16bch_0.0001lr'

file_path = save_path + 'bw_' + name + '.h5'

#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath = file_path,
#    monitor = 'val_accuracy',
#    mode = 'max',
#    save_best_only=True)

#train_summary = embedding_model.fit(train_gen, epochs = 200, verbose = 1, callbacks = [model_checkpoint_callback], validation_data = val_gen, 
#                              shuffle = True, steps_per_epoch = len(train_gen)//batch_size, 
#                              validation_steps = len(val_gen)//batch_size)

embedding_model.load_weights(save_path + 'bw_' + name + '.h5')

In [16]:
embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-3].output) # for embedding model
#embedding_model_0 = Model(inputs = embedding_model.input, outputs = embedding_model.layers[-2].output) # for embedding model variants
embedding_model_0.summary()

trainval_vec = embedding_model_0.predict(trainval_input1)
print(trainval_vec, trainval_vec.shape)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2D)             (None, 1, 1, 2048)        4196352   
                                                                 
 layer2 (Conv2D)             (None, 1, 1, 1024)        2098176   
                                                                 
Total params: 6,294,528
Trainable params: 6,294,528
Non-trainable params: 0
_________________________________________________________________
323/323 [==============================] - 0s 674us/step
[[[[2.6868408  2.4213166  0.         ... 0.         0.
    0.        ]]]


 [[[2.4873016  2.422116   0.         ... 0.         0.37754858
    0.        ]]]


 [[[3.686314   2.920684   0.         ... 0.         0.
    0.        ]]]


 ...


 [[[3.4510417  2

In [17]:
inputt = Input(shape = (1, 1, trainval_attributes.shape[1]))
hidden1 = Conv2DTranspose(1024, 1, name="layer1", activation='relu')(inputt)

embedding_model_1 = Model(inputs = inputt, outputs = hidden1)
embedding_model_1.summary()

w_list = Model(inputs = embedding_model_0.layers[-2].output, outputs = embedding_model_0.output).get_weights()
#print(weights_list2)

embedding_model_1.set_weights([np.transpose(w_list[0], (0, 1, 3, 2)), w_list[1]])

trainval_attributes = embedding_model_1.predict(trainval_input2)
print(trainval_attributes, trainval_attributes.shape)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 layer1 (Conv2DTranspose)    (None, 1, 1, 1024)        2098176   
                                                                 
Total params: 2,098,176
Trainable params: 2,098,176
Non-trainable params: 0
_________________________________________________________________
323/323 [==============================] - 0s 656us/step
[[[[0.09557883 0.         1.3635446  ... 0.4947308  0.1642455
    0.        ]]]


 [[[0.09557883 0.         1.3635446  ... 0.4947308  0.1642455
    0.        ]]]


 [[[0.09557883 0.         1.3635446  ... 0.4947308  0.1642455
    0.        ]]]


 ...


 [[[0.769035   0.72159773 1.7082789  ... 0.         0.
    0.        ]]]


 [[[0.769035   0.72159773 1.7082789  ... 0.         0.
    0

In [18]:

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], 1, 1, attribute_new.shape[1]])

# as labels range from 1 to 717, we have subtract 1
attribute_2 = embedding_model_1.predict(attribute_new)
print(attribute_2, attribute_2.shape)


23/23 [==============================] - 0s 774us/step
[[[[1.1980121  0.         0.         ... 0.         0.
    1.0407351 ]]]


 [[[0.27771    0.         0.         ... 0.13699925 0.24434765
    0.24226362]]]


 [[[0.         0.         0.         ... 0.28185102 0.
    1.1366591 ]]]


 ...


 [[[0.8728378  0.         0.         ... 0.18083176 0.47939357
    0.03623104]]]


 [[[0.24888746 0.         0.         ... 0.         0.04705606
    0.08469189]]]


 [[[0.21459852 0.27802858 0.         ... 0.         0.23480298
    0.        ]]]] (717, 1, 1, 1024)


In [19]:


signature_2 = attribute_2.transpose()
signature_2 = np.reshape(signature_2, [signature_2.shape[0], signature_2.shape[3]])
print(signature_2, signature_2.shape)#(1024, 717)

trainval_sig = signature_2[:, (unique_labels_trainval)-1]
print("Signature for trainval:", trainval_sig.shape)# 1024, 645


[[1.1980121  0.27771    0.         ... 0.8728378  0.24888746 0.21459852]
 [0.         0.         0.         ... 0.         0.         0.27802858]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.13699925 0.28185102 ... 0.18083176 0.         0.        ]
 [0.         0.24434765 0.         ... 0.47939357 0.04705606 0.23480298]
 [1.0407351  0.24226362 1.1366591  ... 0.03623104 0.08469189 0.        ]] (1024, 717)
Signature for trainval: (1024, 645)


In [20]:
# by doing this modification, we are changing the range of trainval and test seen labels from 0 to 644
# and test unseen labels from 0 to 71

k = 0
new_labels_trainval = np.zeros((len(labels_trainval), 1), dtype = 'int')
for labels in unique_labels_trainval:
    new_labels_trainval[labels_trainval == labels] = k
    k = k+1
    
print(new_labels_trainval, new_labels_trainval.shape)#(10320, 1)
print(np.unique(new_labels_trainval), np.unique(new_labels_trainval).shape)


[[43]
 [43]
 [43]
 ...
 [61]
 [61]
 [61]] (10320, 1)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 23

In [21]:
#params for trainval and test set
m_trainval = new_labels_trainval.shape[0]# number of instances in training set: 10320
print(m_trainval)

z_trainval = len(unique_labels_trainval)# number of classes in training set: 645
print(z_trainval)


10320
645


In [22]:
from tensorflow.keras.utils import to_categorical
gt_trainval = to_categorical(new_labels_trainval, z_trainval)

print(gt_trainval, gt_trainval.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (10320, 645)


In [23]:
input1_shape = trainval_vec.shape[3]
print(input1_shape)

attribute_shape = trainval_sig.shape[0]
print(attribute_shape)

output_shape = z_trainval
print(output_shape)


1024
1024
645


In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad

# define model2 for attribute to class label mapping

input2 = Input(shape = (1, 1, attribute_shape))
flat = Flatten()(input2)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model2 = Model(inputs = input2, outputs = output)


opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model2.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 645)               661125    
                                                                 
Total params: 661,125
Trainable params: 661,125
Non-trainable params: 0
_________________________________________________________________


In [25]:
# define model1 for resnet feature to class label mapping

input1 = Input(shape = (1,1, input1_shape))
#inter_pre = Dense(512, name="intermediate_previous", activation='relu')(input1)
inter = Conv1D(attribute_shape, kernel_size = 1, name = "intermediate", activation = 'linear')(input1)
flat = Flatten()(inter)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model1 = Model(inputs = input1, outputs = output)

opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)
#opt = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 1, 1024)]      0         
                                                                 
 intermediate (Conv1D)       (None, 1, 1, 1024)        1049600   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 output (Dense)              (None, 645)               661125    
                                                                 
Total params: 1,710,725
Trainable params: 1,710,725
Non-trainable params: 0
_________________________________________________________________


In [26]:
trainval_input1 = trainval_vec
print(trainval_input1.shape)

trainval_input2 = trainval_attributes
print(trainval_input2.shape)

trainval_output = gt_trainval
print(trainval_output.shape)

(10320, 1, 1, 1024)
(10320, 1, 1, 1024)
(10320, 645)


In [27]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 8
from sklearn.model_selection import train_test_split    


X_train1, X_val1, y_train1, y_val1 = train_test_split(trainval_input1, trainval_output, test_size = 0.2, random_state = 42)

#train_gen1 = DataGenerator(X_train1, y_train1, batch_size)   
#val_gen1 = DataGenerator(X_val1, y_val1, batch_size)


X_train2, X_val2, y_train2, y_val2 = train_test_split(trainval_input2, trainval_output, test_size = 0.2, random_state = 42)

#train_gen2 = DataGenerator(X_train2, y_train2, batch_size)   
#val_gen2 = DataGenerator(X_val2, y_val2, batch_size)

In [28]:
labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 14340 x 1

test_seen_loc = np.squeeze(att_splits['test_seen_loc']-1)
print(np.unique(test_seen_loc), np.max(np.unique(test_seen_loc))) # smallest location: 0, largest location 14335

labels_test_seen = labels[test_seen_loc]
print('labels_test_seen', labels_test_seen, labels_test_seen.shape)

unique_labels_test_seen = np.unique(labels_test_seen) # labels min:1 max:717
print('unique_labels_test_seen', unique_labels_test_seen, unique_labels_test_seen.shape)# 645 classes

test_seen_vec = X_features[:, test_seen_loc].transpose()
print("Features for test seen:", test_seen_vec.shape)# (2580, 2048)

test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
test_seen_vec = embedding_model_0.predict(test_seen_vec)
print(test_seen_vec.shape)# (2580, 1, 1, 1024)


test_seen_attributes = np.zeros((len(test_seen_loc), attribute_2.shape[1], attribute_2.shape[2], attribute_2.shape[3]))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute_2[int(labels_test_seen[i])-1]

print(test_seen_attributes, test_seen_attributes.shape)# (2580, 1, 1, 1024)



test_seen_sig = signature_2[:, (unique_labels_test_seen)-1]
print("Signature for test seen:", test_seen_sig.shape)

labels [[  1]
 [  1]
 [  1]
 ...
 [717]
 [717]
 [717]] (14340, 1)
[    0     3    14 ... 14322 14331 14335] 14335
labels_test_seen [[50]
 [50]
 [50]
 ...
 [70]
 [70]
 [70]] (2580, 1)
unique_labels_test_seen [  1   2   3   5   6   7   8   9  10  12  13  14  15  16  17  18  19  20
  21  22  23  26  27  28  29  30  31  32  34  35  36  37  38  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  55  56  57  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  74  77  78  79  80  81  82  83
  84  85  87  88  89  90  91  92  93  94  95  97  98  99 101 102 103 105
 106 107 108 109 110 111 112 114 115 116 117 118 119 120 121 122 123 124
 126 127 128 129 130 132 133 134 135 136 137 138 140 141 142 143 144 145
 147 148 149 150 151 152 154 155 156 157 158 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 186 187 188 189 190 191 192 193 194 195 196 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 216 218 219 220 22

In [29]:

test_unseen_loc = np.squeeze(att_splits['test_unseen_loc']-1)
print(np.unique(test_unseen_loc), np.max(np.unique(test_unseen_loc))) # smallest location: 60, largest location 14319

labels_test_unseen = labels[test_unseen_loc]
print('labels_test_unseen', labels_test_unseen, labels_test_unseen.shape)

unique_labels_test_unseen = np.unique(labels_test_unseen) # labels min:4 max:716
print('unique_labels_test_unseen', unique_labels_test_unseen, unique_labels_test_unseen.shape)# 72 classes


test_unseen_vec = X_features[:, test_unseen_loc].transpose()
print("Features for test unseen:", test_unseen_vec.shape) #(1440, 2048)


test_unseen_vec = np.reshape(test_unseen_vec, [test_unseen_vec.shape[0], 1, 1, test_unseen_vec.shape[1]])
test_unseen_vec = embedding_model_0.predict(test_unseen_vec)
print(test_unseen_vec.shape)

#test_unseen_attributes = np.zeros((len(test_unseen_loc), attribute_new.shape[1]))
#for i in range(len(test_unseen_loc)):
#    test_unseen_attributes[i] = attribute_new[int(labels_test_unseen[i])-1]

#print(test_unseen_attributes, test_unseen_attributes.shape)# (7913, 85)


#test_unseen_attributes_2 = model0.predict(test_unseen_attributes)
#print(test_unseen_attributes_2, test_unseen_attributes_2.shape)

test_unseen_sig = signature_2[:, (unique_labels_test_unseen)-1]

print("Signature for test unseen:", test_unseen_sig.shape)

[   60    61    62 ... 14317 14318 14319] 14319
labels_test_unseen [[  4]
 [  4]
 [  4]
 ...
 [716]
 [716]
 [716]] (1440, 1)
unique_labels_test_unseen [  4  11  24  25  33  39  54  58  73  75  76  86  96 100 104 113 125 131
 139 146 153 159 185 197 217 222 238 246 247 255 260 263 287 299 316 329
 337 343 354 359 380 382 421 424 426 441 449 472 483 494 509 510 518 530
 559 561 581 623 632 636 646 651 657 659 675 680 682 696 711 712 713 716] (72,)
Features for test unseen: (1440, 2048)
45/45 [==============================] - 0s 894us/step
(1440, 1, 1, 1024)
Signature for test unseen: (1024, 72)


In [30]:

l = 0
new_labels_test_seen = np.zeros((len(labels_test_seen), 1), dtype = 'int')
for labels in unique_labels_test_seen:
    new_labels_test_seen[labels_test_seen == labels] = l
    l = l+1
    
print(new_labels_test_seen, new_labels_test_seen.shape)# (2580, 1)



n_test_seen = new_labels_test_seen.shape[0]# 2580
print(n_test_seen)

z1_test_seen = len(unique_labels_test_seen)# 645
print(z1_test_seen)


m = 0
new_labels_test_unseen = np.zeros((len(labels_test_unseen), 1), dtype = 'int')
for labels in unique_labels_test_unseen:
    new_labels_test_unseen[labels_test_unseen == labels] = m
    m = m+1  

print(new_labels_test_unseen, new_labels_test_unseen.shape) #  (1440, 1)  

n_test_unseen = new_labels_test_unseen.shape[0]# 1440
print(n_test_unseen)

z1_test_unseen = len(unique_labels_test_unseen)# 72
print(z1_test_unseen)


[[43]
 [43]
 [43]
 ...
 [61]
 [61]
 [61]] (2580, 1)
2580
645
[[ 0]
 [ 0]
 [ 0]
 ...
 [71]
 [71]
 [71]] (1440, 1)
1440
72


In [31]:
iteration = 5
epochs1 = 200
epochs2 = 200

best_performance_micro = [0, 0, 0]
best_performance_macro = [0, 0, 0]

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/SUN/With_LE/'
name = 'model1_conv_SUN_LE_GPT2attT_1024_lin_it5_200eph_adam_cce_16bch_1e-2lr_model2_adam_lr-2_200'  
   
for i in range(iteration):
    X_train2_it = X_train2[(len(X_train2)//iteration)*i:(len(X_train2)//iteration)*(i+1)]
    X_val2_it = X_val2[(len(X_val2)//iteration)*i:(len(X_val2)//iteration)*(i+1)]
    y_train2_it = y_train2[(len(y_train2)//iteration)*i:(len(y_train2)//iteration)*(i+1)]
    y_val2_it = y_val2[(len(y_val2)//iteration)*i:(len(y_val2)//iteration)*(i+1)]
    
    train_gen2 = DataGenerator(X_train2_it, y_train2_it, batch_size)   
    val_gen2 = DataGenerator(X_val2_it, y_val2_it, batch_size)

    train_summary2 = model2.fit(train_gen2, epochs = epochs2, verbose = 0, callbacks = None, validation_data = val_gen2, 
                              shuffle = True, steps_per_epoch = len(train_gen2)//batch_size, 
                              validation_steps = len(val_gen2)//batch_size)

    print("iteration:", i)
    print('model 2 is trained:', 'training acc:', train_summary2.history['accuracy'][-1], ',',  
          'training loss:', train_summary2.history['loss'][-1], ',', 
          'validation acc:', train_summary2.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary2.history['val_loss'][-1])

    weights_list2 = model2.get_weights()
    #print(weights_list2)

    model1.layers[-1].set_weights(weights_list2)

    X_train1_it = X_train1[(len(X_train1)//iteration)*i:(len(X_train1)//iteration)*(i+1)]
    X_val1_it = X_val1[(len(X_val1)//iteration)*i:(len(X_val1)//iteration)*(i+1)]
    y_train1_it = y_train1[(len(y_train1)//iteration)*i:(len(y_train1)//iteration)*(i+1)]
    y_val1_it = y_val1[(len(y_val1)//iteration)*i:(len(y_val1)//iteration)*(i+1)]
    
    train_gen1 = DataGenerator(X_train1_it, y_train1_it, batch_size)   
    val_gen1 = DataGenerator(X_val1_it, y_val1_it, batch_size)
    
    for layer in model1.layers[2:]:
        layer.trainable = True

    train_summary1 = model1.fit(train_gen1, epochs = epochs1, verbose = 0, callbacks = None, validation_data = val_gen1, 
                              shuffle = True, steps_per_epoch = len(train_gen1)//batch_size, 
                              validation_steps = len(val_gen1)//batch_size)
    
    #print("iteration:", i)
    print('model 1 is trained:', 'training acc:', train_summary1.history['accuracy'][-1], ',',  
          'training loss:', train_summary1.history['loss'][-1], ',', 
          'validation acc:', train_summary1.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary1.history['val_loss'][-1])
    
    weights_list1 = Model(inputs = model1.input, outputs = model1.layers[-1].output).get_weights()
    
    #predictions
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, np.matmul(weights_list1[0], weights_list1[2])), test_seen_sig)
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0]), test_seen_sig)
    
    outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0][0]), test_seen_sig)
    preds_seen = np.array([np.argmax(output) for output in outputs_seen])
    
    cm_seen = confusion_matrix(new_labels_test_seen, preds_seen)
    #print(cm)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_seen_micro = cm_seen.astype('float') / cm_seen.sum(axis=1)[:, np.newaxis]
    #print(cm)
    avg_seen_micro = (sum(cm_seen_micro.diagonal())/len(unique_labels_test_seen))*100

    avg_seen_macro = (sum(cm_seen.diagonal())/len(new_labels_test_seen))*100
    
    #predictions
    #outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0]), test_unseen_sig)
    outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0][0]), test_unseen_sig)
    
    preds_unseen = np.array([np.argmax(output) for output in outputs_unseen])
    
    cm_unseen = confusion_matrix(new_labels_test_unseen, preds_unseen)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_unseen_micro = cm_unseen.astype('float') / cm_unseen.sum(axis=1)[:, np.newaxis]
    avg_unseen_micro = (sum(cm_unseen_micro.diagonal())/len(unique_labels_test_unseen))*100

    avg_unseen_macro = (sum(cm_unseen.diagonal())/len(new_labels_test_unseen))*100
    
    harmonic_micro = (2*avg_seen_micro*avg_unseen_micro) / (avg_seen_micro + avg_unseen_micro)
    harmonic_macro = (2*avg_seen_macro*avg_unseen_macro) / (avg_seen_macro + avg_unseen_macro)
    
    print('micro average')
    print('seen accuracy:', avg_seen_micro, 'unseen accuracy:', avg_unseen_micro, 'harmonic mean:', harmonic_micro)
    
    print('macro average')
    print('seen accuracy:', avg_seen_macro, 'unseen accuracy:', avg_unseen_macro, 'harmonic mean:', harmonic_macro)
    
    if harmonic_micro > best_performance_micro[2]:
        best_performance_micro = [avg_seen_micro, avg_unseen_micro, harmonic_micro]
        model1.save_weights(save_path + 'bw_micro_' + name + '.h5', overwrite=True)
        
    if harmonic_macro > best_performance_macro[2]:
        best_performance_macro = [avg_seen_macro, avg_unseen_macro, harmonic_macro]
        model1.save_weights(save_path + 'bw_macro_' + name + '.h5', overwrite=True)
        
    print('best accuracy micro','seen accuracy:', best_performance_micro[0], 'unseen accuracy:', best_performance_micro[1], 'harmonic mean:', best_performance_micro[2])
    print('best accuracy macro', 'seen accuracy:', best_performance_macro[0], 'unseen accuracy:', best_performance_macro[1], 'harmonic mean:', best_performance_macro[2])
    
    print('-----------------------------------------------------------------------------------------------------------')
    weights_list3 = model1.get_weights()
    model2.set_weights(weights_list3[2:])

iteration: 0
model 2 is trained: training acc: 0.9950000047683716 , training loss: 0.014873392879962921 , validation acc: 0.9791666865348816 , validation_loss: 0.26350048184394836
model 1 is trained: training acc: 0.8650000095367432 , training loss: 28.153213500976562 , validation acc: 0.0833333358168602 , validation_loss: 641.3921508789062
micro average
seen accuracy: 11.744186046511627 unseen accuracy: 32.56944444444445 harmonic mean: 17.263384234119318
macro average
seen accuracy: 11.744186046511627 unseen accuracy: 32.56944444444444 harmonic mean: 17.263384234119318
best accuracy micro seen accuracy: 11.744186046511627 unseen accuracy: 32.56944444444445 harmonic mean: 17.263384234119318
best accuracy macro seen accuracy: 11.744186046511627 unseen accuracy: 32.56944444444444 harmonic mean: 17.263384234119318
-----------------------------------------------------------------------------------------------------------
iteration: 1
model 2 is trained: training acc: 1.0 , training loss: 0

In [32]:
gt_test_seen = to_categorical(new_labels_test_seen, z1_test_seen)

print(gt_test_seen, gt_test_seen.shape)

#test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
res1 = model1.evaluate(test_seen_vec, gt_test_seen)

p1 = model1.predict(test_seen_vec, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p1).numpy())

#test_seen_attributes = np.reshape(train_attributes, [test_seen_attributes.shape[0], 1, train_attributes.shape[1]])
#test_seen_attributes = np.reshape(test_seen_attributes, [test_seen_attributes.shape[0], 1, 1, test_seen_attributes.shape[1]])

res2 = model2.evaluate(test_seen_attributes, gt_test_seen)

p2 = model2.predict(test_seen_attributes, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p2).numpy())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (2580, 645)
81/81 [==============================] - 0s 3ms/step - loss: 446.8889 - accuracy: 0.2601
cce =  11.863797
81/81 [==============================] - 0s 3ms/step - loss: 5.5378 - accuracy: 0.4775
cce =  4.837074


In [35]:

accuracy_seen_updated = res1[1]*100
unseen_accuracy = 40.35
h = (2*accuracy_seen_updated*unseen_accuracy) / (accuracy_seen_updated + unseen_accuracy)
print(h)


accuracy_seen_updated2 = ((res1[1]*100)+(res2[1]*100))/2
print(accuracy_seen_updated2)
h = (2*accuracy_seen_updated2*unseen_accuracy) / (accuracy_seen_updated2 + unseen_accuracy)
print(h)

31.628942451082967
36.87984496355057
38.53696054890673


In [34]:
from sklearn.metrics import precision_recall_fscore_support

pp1 = np.array([np.argmax(output) for output in p1])
pp2 = np.array([np.argmax(output) for output in p2])

seen_macro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'macro')
seen_macro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'macro')
print('precision_seen_macro', (seen_macro1[0] + seen_macro2[0])/2, 'recall_seen_macro', (seen_macro1[1] + seen_macro2[1])/2, 'f1_seen_macro', (seen_macro1[2] + seen_macro2[2])/2)


seen_micro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'micro')
seen_micro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'micro')
print('precision_seen_micro', (seen_micro1[0] + seen_micro2[0])/2, 'recall_seen_micro', (seen_micro1[1] + seen_micro2[1])/2, 'f1_seen_micro', (seen_micro1[2] + seen_micro2[2])/2)

unseen_macro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'macro')
unseen_micro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'micro')

print('precision_unseen_macro', unseen_macro[0], 'recall_unseen_macro', unseen_macro[1], 'f1_unseen_macro', unseen_macro[2])
print('precision_unseen_micro', unseen_micro[0], 'recall_unseen_micro', unseen_micro[1], 'f1_unseen_micro', unseen_micro[2])


precision_seen_macro 0.33850322490579743 recall_seen_macro 0.3687984496124031 f1_seen_macro 0.3159834378364652
precision_seen_micro 0.3687984496124031 recall_seen_micro 0.3687984496124031 f1_seen_micro 0.3687984496124031
precision_unseen_macro 0.44589130246972236 recall_unseen_macro 0.4034722222222222 f1_unseen_macro 0.35512882372829324
precision_unseen_micro 0.40347222222222223 recall_unseen_micro 0.40347222222222223 f1_unseen_micro 0.40347222222222223


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
